# Chapter 10 - 시퀀스 해킹, 해시, 슬라이스

* 기본 시퀀스 프로토콜: \_\_len()\_\_과 \_\_gettiem\_\_() 메서드
* 여러 항목을 가진 객체를 안전하게 표현
* 슬라이싱을 지원해서 새로운 벡터 객체 생성
* 포함된 요소 값을 모두 고려한 집합 해싱
* 커스터마이즈된 포맷 언어 확장

## 10.1 Vector: 사용자 정의 시퀀스형

## 10.2 Vector 버전 #1: Vector2d 호환

In [1]:
from array import array
import reprlib
import math

class Vector:
    typecode = 'd' # double
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self): ## 디버깅용으로 많이 사용한다고 함.
        components = reprlib.repr(self._components)
        # print(components)
        # print(components.find('['))
        components = components[components.find('['):-1]
        # print(components)
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__ (self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [2]:
Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [4]:
Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [5]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])


## 10.3 프로토콜과 덕 타이핑

In [8]:
import collections

Card = collections.namedtuple('Card', ['rank', 
                                       'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

이 클래스가 시퀀스처럼 동작하기 때문에 시퀀스인 것이다.

## 10.4 Vector 버전 #2: 슬라이스 가능한 시퀀스

In [10]:
from array import array
import reprlib
import math

class Vector:
    typecode = 'd' # double
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        # print(components)
        # print(components.find('['))
        components = components[components.find('['):-1]
        # print(components)
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__ (self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        return self._components[index]

In [12]:
v1 = Vector([3, 4, 5])
len(v1)

3

In [13]:
v7 = Vector(range(7)); v7

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [14]:
v7[1:4]

array('d', [1.0, 2.0, 3.0])

## 10.4.1 슬라이싱의 작동 방식

In [15]:
class MySeq:
    def __getitem__(self, index):
        return index

In [16]:
s = MySeq()

In [17]:
s[1]

1

In [18]:
s[1:4]

slice(1, 4, None)

In [19]:
s[1:4:2]

slice(1, 4, 2)

In [20]:
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [21]:
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

In [22]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [23]:
help(slice.indices)

Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



In [26]:
slice(None, 10, 2).indices(5)

(0, 5, 2)

'ABCDE'[:10:2]는 'ABCDE'[0:5:2] 와 동일하다. 

In [25]:
slice(-3, None, None).indices(5)

(2, 5, 1)

'ABCDE'[-3:]는 'ABCDE'[2:5:1] 와 동일하다.

## 10.4.2 슬라이스를 인식하는 \_\_getitem\_\_()

In [27]:
from array import array
import reprlib
import math
import numbers # 추가

class Vector:
    typecode = 'd' # double
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        # print(components)
        # print(components.find('['))
        components = components[components.find('['):-1]
        # print(components)
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__ (self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self) # Vector type
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

In [28]:
v7 = Vector(range(7))

In [29]:
v7[-1]

6.0

In [30]:
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [31]:
v7[-1:]

Vector([6.0])

In [32]:
v7[1, 2]

TypeError: Vector indices must be integers

## 10.5 Vector 버전 #3: 동적 속성 접근